In [1]:
from getpass import getpass

from sympy.physics.units import temperature

ACCESS_TOKEN = getpass('YOUR_GITHUB_PERSONAL_ACCESS_TOKEN')

In [2]:
from langchain.document_loaders import GitHubIssuesLoader

loader = GitHubIssuesLoader(
    repo = 'YOUR_USERNAME/YOUR_REPO',
    access_token = ACCESS_TOKEN,
    include_prs=False,
    state='all'
)

In [3]:
docs = loader.load()

In [4]:
len(docs)

12

In [5]:
# Git issues can be longer than the context window of the model so we will make chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=20)
chunked_docs = splitter.split_documents(docs)

# Now creating Embeddings + Retreivers

In [7]:
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
data_base = FAISS.from_documents(chunked_docs, SentenceTransformerEmbeddings(model_name = 'all-MiniLM-L6-v2'))

In [8]:
retriever = data_base.as_retriever(
    search_type = 'similarity', # perform similarity search between document and query
    search_kwargs= {'k':4} # Return top 4 results
)

# Load Quantized Model

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
checkpoint = 'google/gemma-2-2b'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [10]:
model = AutoModelForCausalLM.from_pretrained(checkpoint, config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# Now setup the LLM Chain

In [12]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import  pipeline
from langchain_core.output_parsers import StrOutputParser

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [16]:
text_generation_pipeline = pipeline(
    model = model,
    tokenizer = tokenizer,
    task = 'text-generation',
    temperature = 0.3,
    do_sample = True,
    repetition_penalty = 1.1,
    return_full_text = True,
    max_new_tokens = 400,
    device = device
)

llm = HuggingFacePipeline(pipeline = text_generation_pipeline)

In [17]:
prompt_template = """
<start_of_turn>system
You are a helpful assistant. Answer the user's question truthfully based on the context.
If the answer is not in the context, say "I don't have enough information".
Use the context below:

{context}
<end_of_turn>

<start_of_turn>user
{question}
<end_of_turn>

<start_of_turn>model
"""

prompt = PromptTemplate(
    input_variables=['context', 'context'],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

In [18]:
from langchain_core.runnables import RunnablePassthrough
retriever = data_base.as_retriever()

rag_chain = (
    {'context':retriever, 'question':RunnablePassthrough()} | llm_chain
)

In [22]:
question = 'What should I do if the login page keeps showing Invalid Credentials even when my password is correct?'

In [23]:
llm_chain.invoke({'context':'', 'question':question})

'\n<start_of_turn>system\nYou are a helpful assistant. Answer the user\'s question truthfully based on the context.\nIf the answer is not in the context, say "I don\'t have enough information".\nUse the context below:\n\n\n<end_of_turn>\n\n<start_of_turn>user\nWhat should I do if the login page keeps showing Invalid Credentials even when my password is correct?\n<end_of_turn>\n\n<start_of_turn>model\nThe system has detected that your credentials are invalid. Please try again later or contact support for assistance.\n مرئيه (1) {\n  System.out.println("Invalid Credentials");\n}'

In [24]:
rag_chain.invoke(question)

'\n<start_of_turn>system\nYou are a helpful assistant. Answer the user\'s question truthfully based on the context.\nIf the answer is not in the context, say "I don\'t have enough information".\nUse the context below:\n\n[Document(id=\'2685d26d-9e27-4934-9f1f-014ea1223ac7\', metadata={\'url\': \'https://github.com/DK097432/LearningLLMs/issues/4\', \'title\': \'Login page crashes on invalid credentials\', \'creator\': \'DK097432\', \'created_at\': \'2025-11-01T21:56:18Z\', \'comments\': 0, \'state\': \'open\', \'labels\': [], \'assignee\': None, \'milestone\': None, \'locked\': False, \'number\': 4, \'is_pull_request\': False}, page_content=\'When a user enters an invalid password twice, the login page crashes and returns a 500 internal server error instead of showing validation message.\'), Document(id=\'d03e4459-b89d-4ee7-bf05-b6e1dd64429f\', metadata={\'url\': \'https://github.com/DK097432/LearningLLMs/issues/1\', \'title\': \'Bug: Login failure on Chrome\', \'creator\': \'DK097432\'